To run this notebook SentenceTransformer needed to be installed

In [1]:
#!pip install SentenceTransformer

## Import Libraries

In [2]:
'''basics'''
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import vectorize_embed as em
import pandas as pd
#import pickle5 as pickle
import pickle
import numpy as np

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB


'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, multi_labeled
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

import joblib

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## Import data

In [3]:
df = pd.read_csv('../../data/processed/encoded_labels/_hot_topics.csv')
df.columns

Index(['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', '_hot_topics',
       'Unnamed: 4', 'cov_19', 'crisis_setting', 'digital_transformation',
       'food_and_agricultural_commodities_strategy', 'green_recovery',
       'health', 'human_rights', 'leaving_no_one_behind',
       'multi_stakeholder_collaboration', 'nature_based_solution', 'no tag',
       'plastic', 'poverty_reduction', 'public_private_partnership', 'sids',
       'south_south_cooperation', 'structural_system_transformation'],
      dtype='object')

# A recap of previous comparison between different classifiers using TF-IDF

<table>
<thead>
<tr><th>Category                                  </th><th>#Inputs      </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>Unnamed: 4                                </td><td>34</td><td>AdaBoost    
 Stochastic Gradient Descent    
 Random Forest    
 K Nearest Neighbor    
 Gaussian Naive Bayes    
 Decsision Tree              </td><td>0.9388888888888889    
 0.9555555555555556    
 0.9555555555555556    
 0.9555555555555556    
 0.8500000000000000    
 0.8888888888888888                 </td><td>0.646551724137931   
 0.9776536312849162   
 0.9776536312849162   
 0.9776536312849162   
 0.5233026467203682   
 0.47337278106508873                  </td><td>0.5994152046783625    
 0.5555555555555556    
 0.5555555555555556    
 0.5555555555555556    
 0.5526315789473684    
 0.4678362573099415               </td><td>0.6173913043478261    
 0.5885714285714286    
 0.5885714285714286    
 0.5885714285714286    
 0.5234826943818021    
 0.47058823529411764           </td></tr>
<tr><td>cov_19                                    </td><td>8 </td><td>Decsision Tree    
 Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 K Nearest Neighbor              </td><td>0.9888888888888889    
 0.9888888888888889    
 0.9833333333333333    
 0.9833333333333333    
 0.9833333333333333    
 0.9833333333333333                 </td><td>0.8305084745762712   
 0.8305084745762712   
 0.4916666666666666   
 0.4916666666666666   
 0.4916666666666666   
 0.4916666666666666                  </td><td>0.8305084745762712    
 0.8305084745762712    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000               </td><td>0.8305084745762712    
 0.8305084745762712    
 0.49579831932773105    
 0.49579831932773105    
 0.49579831932773105    
 0.49579831932773105           </td></tr>
<tr><td>crisis_setting                            </td><td>2</td><td>Stochastic Gradient Descent    
 Random Forest    
 Decsision Tree    
 AdaBoost    
 Gaussian Naive Bayes    
 K Nearest Neighbor              </td><td>0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445                 </td><td>0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223                  </td><td>0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000               </td><td>0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329           </td></tr>
<tr><td>digital_transformation                    </td><td>3</td><td>Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 Gaussian Naive Bayes    
 K Nearest Neighbor    
 Decsision Tree              </td><td>0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9944444444444445    
 0.9833333333333333                 </td><td>0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49722222222222223   
 0.49719101123595505                  </td><td>0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.49441340782122906               </td><td>0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4986072423398329    
 0.4957983193277311           </td></tr>
<tr><td>food_and_agricultural_commodities_strategy</td><td>245</td><td>Stochastic Gradient Descent    
 Random Forest    
 K Nearest Neighbor    
 Gaussian Naive Bayes    
 Decsision Tree    
 AdaBoost              </td><td>0.7166666666666667    
 0.7055555555555556    
 0.6888888888888889    
 0.6722222222222223    
 0.6666666666666666    
 0.6666666666666666                 </td><td>0.7197786804874632   
 0.7092426187419769   
 0.6852678571428572   
 0.6885714285714286   
 0.660633484162896   
 0.6608297413793103                  </td><td>0.697209653092006    
 0.6843891402714932    
 0.6877828054298643    
 0.6866515837104072    
 0.660633484162896    
 0.6500754147812972               </td><td>0.6995188373539327    
 0.6859259259259259    
 0.6857463524130192    
 0.6721311475409836    
 0.660633484162896    
 0.6511177154671146           </td></tr>
<tr><td>green_recovery                            </td><td>9</td><td>Decsision Tree    
 Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 K Nearest Neighbor              </td><td>0.9944444444444445    
 0.9777777777777777    
 0.9722222222222222    
 0.9888888888888889    
 0.9888888888888889    
 0.9888888888888889                 </td><td>0.9972067039106145   
 0.6666666666666666   
 0.5971428571428572   
 0.49444444444444446   
 0.49444444444444446   
 0.49444444444444446                  </td><td>0.7500000000000000    
 0.9887640449438202    
 0.7387640449438202    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000               </td><td>0.8319327731092436    
 0.7443181818181819    
 0.6357749898826387    
 0.49720670391061456    
 0.49720670391061456    
 0.49720670391061456           </td></tr>
<tr><td>health                                    </td><td>22</td><td>K Nearest Neighbor    
 Decsision Tree    
 Gaussian Naive Bayes    
 AdaBoost    
 Stochastic Gradient Descent    
 Random Forest              </td><td>0.9777777777777777    
 0.9555555555555556    
 0.9444444444444444    
 0.9555555555555556    
 0.9611111111111111    
 0.9611111111111111                 </td><td>0.8914285714285715   
 0.7027250206440958   
 0.6549707602339181   
 0.6857142857142857   
 0.7331460674157304   
 0.48055555555555557                  </td><td>0.782824112303881    
 0.7027250206440958    
 0.6969446738232865    
 0.634186622625929    
 0.5685383980181667    
 0.5000000000000000               </td><td>0.8275862068965516    
 0.7027250206440958    
 0.6729651162790697    
 0.6551724137931034    
 0.6011396011396012    
 0.49008498583569404           </td></tr>
<tr><td>human_rights                              </td><td>16</td><td>Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 Random Forest    
 K Nearest Neighbor    
 AdaBoost    
 Decsision Tree              </td><td>0.9833333333333333    
 0.9777777777777777    
 0.9777777777777777    
 0.9777777777777777    
 0.9722222222222222    
 0.9611111111111111                 </td><td>0.9915730337078652   
 0.9888268156424581   
 0.9888268156424581   
 0.9888268156424581   
 0.7387640449438202   
 0.6136363636363636                  </td><td>0.7000000000000000    
 0.6000000000000000    
 0.6000000000000000    
 0.6000000000000000    
 0.5971428571428572    
 0.5914285714285714               </td><td>0.7814649939295832    
 0.6610169491525424    
 0.6610169491525424    
 0.6610169491525424    
 0.6357749898826387    
 0.6011396011396012           </td></tr>
<tr><td>leaving_no_one_behind                     </td><td>18</td><td>Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 K Nearest Neighbor    
 Decsision Tree              </td><td>0.9722222222222222    
 0.9833333333333333    
 0.9833333333333333    
 0.9833333333333333    
 0.9833333333333333    
 0.9777777777777777                 </td><td>0.6942857142857143   
 0.9916201117318435   
 0.9916201117318435   
 0.9916201117318435   
 0.9916201117318435   
 0.7415730337078652                  </td><td>0.7414772727272727    
 0.6250000000000000    
 0.6250000000000000    
 0.6250000000000000    
 0.6250000000000000    
 0.6221590909090908               </td><td>0.7150997150997151    
 0.6957746478873239    
 0.6957746478873239    
 0.6957746478873239    
 0.6957746478873239    
 0.6610169491525424           </td></tr>
<tr><td>multi_stakeholder_collaboration           </td><td>199</td><td>K Nearest Neighbor    
 Stochastic Gradient Descent    
 Gaussian Naive Bayes    
 Random Forest    
 AdaBoost    
 Decsision Tree              </td><td>0.8222222222222222    
 0.8111111111111111    
 0.7944444444444444    
 0.7944444444444444    
 0.7777777777777778    
 0.7111111111111111                 </td><td>0.8236177884615384   
 0.8000000000000000   
 0.7836970474967908   
 0.8091156000679003   
 0.7793640493186242   
 0.6883081896551724                  </td><td>0.7862838915470494    
 0.7870813397129186    
 0.7643540669856459    
 0.7420255183413078    
 0.7288676236044657    
 0.6858054226475279               </td><td>0.798291077181678    
 0.7924297924297925    
 0.7715344242050015    
 0.7565700500749296    
 0.7407834101382489    
 0.6869565217391305           </td></tr>
<tr><td>nature_based_solution                     </td><td>92</td><td>Stochastic Gradient Descent    
 AdaBoost    
 Gaussian Naive Bayes    
 K Nearest Neighbor    
 Decsision Tree    
 Random Forest              </td><td>0.8277777777777777    
 0.8666666666666667    
 0.7277777777777777    
 0.8444444444444444    
 0.7777777777777778    
 0.8666666666666667                 </td><td>0.6605389358300982   
 0.7314974182444062   
 0.6397993311036789   
 0.6608231707317074   
 0.5733333333333334   
 0.8096590909090908                  </td><td>0.6755744255744256    
 0.6343656343656343    
 0.760989010989011    
 0.6053946053946054    
 0.5824175824175823    
 0.5544455544455544               </td><td>0.6673621460506707    
 0.6625000000000000    
 0.6396911638547327    
 0.6226415094339623    
 0.5770676691729323    
 0.5636363636363636           </td></tr>
<tr><td>no tag                                    </td><td>85</td><td>K Nearest Neighbor    
 Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 AdaBoost    
 Random Forest    
 Decsision Tree              </td><td>0.8833333333333333    
 0.7777777777777778    
 0.8666666666666667    
 0.8277777777777777    
 0.8666666666666667    
 0.7722222222222223                 </td><td>0.9393063583815029   
 0.6377005347593583   
 0.788235294117647   
 0.641465175027066   
 0.9318181818181819   
 0.5814029010608357                  </td><td>0.625000000000000    
 0.6936090225563909    
 0.6151315789473684    
 0.5921052631578947    
 0.5714285714285714    
 0.5883458646616542               </td><td>0.6676923076923077    
 0.6527777777777777    
 0.6469434455704478    
 0.6063492063492064    
 0.5884146341463414    
 0.5844360605889971           </td></tr>
<tr><td>plastic                                   </td><td>18</td><td>Gaussian Naive Bayes    
 Decsision Tree    
 Random Forest    
 K Nearest Neighbor    
 AdaBoost    
 Stochastic Gradient Descent              </td><td>0.9722222222222222    
 0.9666666666666667    
 0.9777777777777777    
 0.9777777777777777    
 0.9722222222222222    
 0.9666666666666667                 </td><td>0.7442528735632183   
 0.6914285714285715   
 0.9888268156424581   
 0.9888268156424581   
 0.7387640449438202   
 0.655367231638418                  </td><td>0.7914285714285714    
 0.6914285714285715    
 0.6000000000000000    
 0.6000000000000000    
 0.5971428571428572    
 0.5942857142857143               </td><td>0.7655639489450377    
 0.6914285714285715    
 0.6610169491525424    
 0.6610169491525424    
 0.6357749898826387    
 0.6164772727272727           </td></tr>
<tr><td>poverty_reduction                         </td><td>153</td><td>K Nearest Neighbor    
 Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 Decsision Tree    
 Gaussian Naive Bayes              </td><td>0.7777777777777778    
 0.7888888888888889    
 0.7666666666666667    
 0.7222222222222222    
 0.6833333333333333    
 0.6000000000000000                 </td><td>0.7051671732522796   
 0.7233115468409586   
 0.6806978676266962   
 0.6108374384236454   
 0.5852272727272727   
 0.6262626262626263                  </td><td>0.711111111111111    
 0.6518518518518519    
 0.6074074074074074    
 0.5925925925925926    
 0.5888888888888889    
 0.6666666666666667               </td><td>0.707981829980532    
 0.6701388888888888    
 0.6192586623690572    
 0.5982142857142858    
 0.5868068140630663    
 0.5833333333333335           </td></tr>
<tr><td>public_private_partnership                </td><td>70</td><td>AdaBoost    
 Gaussian Naive Bayes    
 Decsision Tree    
 Stochastic Gradient Descent    
 K Nearest Neighbor    
 Random Forest              </td><td>0.8611111111111112    
 0.7611111111111111    
 0.8111111111111111    
 0.8666666666666667    
 0.8666666666666667    
 0.8777777777777778                 </td><td>0.5976764199655766   
 0.5255842062852538   
 0.5211141060197664   
 0.5194054500412882   
 0.5194054500412882   
 0.4463276836158192                  </td><td>0.5742072572736188    
 0.5415168355671788    
 0.5230467473030402    
 0.5076822491010134    
 0.5076822491010134    
 0.4906832298136646               </td><td>0.5829858215179317    
 0.5243070493516072    
 0.5218750000000000    
 0.5025333947489636    
 0.5025333947489636    
 0.46745562130177515           </td></tr>
<tr><td>sids                                      </td><td>31</td><td>Gaussian Naive Bayes    
 Decsision Tree    
 Stochastic Gradient Descent    
 Random Forest    
 AdaBoost    
 K Nearest Neighbor              </td><td>0.90000000000000    
 0.9388888888888889    
 0.9611111111111111    
 0.9611111111111111    
 0.9611111111111111    
 0.9611111111111111                 </td><td>0.6300345224395858   
 0.6255161023947151   
 0.9804469273743017   
 0.9804469273743017   
 0.9804469273743017   
 0.9804469273743017                  </td><td>0.8284883720930232    
 0.6104651162790697    
 0.5625000000000000    
 0.5625000000000000    
 0.5625000000000000    
 0.5625000000000000               </td><td>0.6727272727272727    
 0.6173913043478261    
 0.6011396011396011    
 0.6011396011396011    
 0.6011396011396011    
 0.6011396011396011           </td></tr>
<tr><td>south_south_cooperation                   </td><td>14</td><td>AdaBoost    
 Gaussian Naive Bayes    
 Stochastic Gradient Descent    
 Random Forest    
 K Nearest Neighbor    
 Decsision Tree              </td><td>0.9833333333333333    
 0.9444444444444444    
 0.9888888888888889    
 0.9888888888888889    
 0.9888888888888889    
 0.9611111111111111                 </td><td>0.6638418079096046   
 0.5833333333333334   
 0.49444444444444446   
 0.49444444444444446   
 0.49444444444444446   
 0.4942857142857143                  </td><td>0.7443820224719101    
 0.9719101123595506    
 0.5000000000000000    
 0.5000000000000000    
 0.5000000000000000    
 0.4859550561797753               </td><td>0.6957746478873239    
 0.6284062758051197    
 0.49720670391061456    
 0.49720670391061456    
 0.49720670391061456    
 0.49008498583569404           </td></tr>
<tr><td>structural_system_transformation          </td><td>85</td><td>K Nearest Neighbor    
 Stochastic Gradient Descent    
 Gaussian Naive Bayes    
 AdaBoost    
 Random Forest    
 Decsision Tree              </td><td>0.8888888888888888    
 0.8277777777777777    
 0.8111111111111111    
 0.8555555555555555    
 0.9000000000000000    
 0.7777777777777778                 </td><td>0.7108433734939759   
 0.6277332756007794   
 0.6215277777777778   
 0.6234567901234568   
 0.949438202247191   
 0.5464527027027026                  </td><td>0.653125    
 0.6843750000000000    
 0.6968750000000000    
 0.6125000000000000    
 0.5500000000000000    
 0.5687500000000000               </td><td>0.6752075063154096    
 0.6459166190748143    
 0.6405075187969925    
 0.6175220660346519    
 0.5642818719741797    
 0.5504495504495505           </td></tr>
</tbody>
</table>

# Compare different embeddings performances

In [4]:
categories = ['Unnamed: 4', 'cov_19', 'crisis_setting', 'digital_transformation',
       'food_and_agricultural_commodities_strategy', 'green_recovery',
       'health', 'human_rights', 'leaving_no_one_behind',
       'multi_stakeholder_collaboration', 'nature_based_solution', 'no tag',
       'plastic', 'poverty_reduction', 'public_private_partnership', 'sids',
       'south_south_cooperation', 'structural_system_transformation']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()


## Train multiple Embeddings with SGD / OneVsRest multi-label strategy

In [5]:
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():  
        print ('processing :' + str(k))
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)  
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df, model

      

for category in categories:

    dff, model = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    
#from tabulate import tabulate  
#header = lis[0].keys()
#rows =  [x.values() for x in lis]


processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
processing :Bert
processing :Glove
processing :Distilbert
processing :Roberta
pro

In [8]:
from tabulate import tabulate  
header = lis[0].keys()
rows =  [x.values() for x in lis]
print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>Unnamed: 4                                </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.9337016574585635    
 0.9558011049723757    
 0.9502762430939227    
 0.9392265193370166                 </td><td>0.6076589595375723   
 0.47790055248618785   
 0.4777777777777778   
 0.47752808988764045                  </td><td>0.6076589595375723    
 0.5    
 0.49710982658959535    
 0.4913294797687861               </td><td>0.6076589595375723    
 0.4887005649717514    
 0.48725212464589235    
 0.4843304843304843           </td></tr>
<tr><td>cov_19                                    </td><td>Roberta    
 Bert    
 Distilbert    
 Glove              </td><td>0.9834254143646409    
 0.9779005524861878    
 0.9723756906077348    
 0.9668508287292817    

### copy-past the output of the last cell here

<table>
<thead>
<tr><th>Category                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>Unnamed: 4                                </td><td>Roberta    
 Glove  <br>  
 Bert    <br>
 Distilbert              </td><td>0.9337016574585635    
 0.9558011049723757    
 0.9502762430939227    
 0.9226519337016574                 </td><td>0.6076589595375723   
 0.47790055248618785   
 0.4777777777777778   
 0.47714285714285715                  </td><td>0.6076589595375723   <br> 
 0.5    <br>
 0.49710982658959535    
 0.48265895953757226               </td><td>0.6076589595375723    
 0.4887005649717514    
 0.48725212464589235    
 0.47988505747126436           </td></tr>
<tr><td>cov_19                                    </td><td>Glove    
 Roberta <br>   
 Bert   <br> 
 Distilbert              </td><td>0.988950276243094    
 0.9834254143646409    
 0.9779005524861878    
 0.9723756906077348                 </td><td>0.494475138121547   
 0.49444444444444446   
 0.49441340782122906   
 0.4943820224719101                  </td><td>0.5 <br>   
 0.4972067039106145    
 0.49441340782122906    
 0.49162011173184356               </td><td>0.49722222222222223    
 0.4958217270194986    
 0.49441340782122906    
 0.4929971988795518           </td></tr>
<tr><td>crisis_setting                            </td><td>Distilbert    
 Roberta <br>   
 Bert  <br>  
 Glove              </td><td>1.0   <br> 
 1.0   <br> 
 0.994475138121547    
 0.9613259668508287                 </td><td>1.0  <br> 
 1.0   <br>
 0.5   <br>
 0.5                  </td><td>1.0    <br>
 1.0    <br>
 0.4972375690607735    
 0.48066298342541436               </td><td>1.0 <br>   
 1.0    <br>
 0.4986149584487534    
 0.49014084507042255           </td></tr>
<tr><td>digital_transformation                    </td><td>Distilbert    
 Roberta  <br>  
 Bert   <br> 
 Glove              </td><td>0.988950276243094    
 0.988950276243094    
 0.988950276243094    
 0.9779005524861878                 </td><td>0.494475138121547   
 0.494475138121547   
 0.494475138121547   
 0.49441340782122906                  </td><td>0.5   <br> 
 0.5    <br>
 0.5    <br>
 0.49441340782122906               </td><td>0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223    
 0.49441340782122906           </td></tr>
<tr><td>food_and_agricultural_commodities_strategy</td><td>Roberta    
 Bert    <br>
 Distilbert  <br>  
 Glove     <br>         </td><td>0.6353591160220995    
 0.6077348066298343    
 0.6408839779005525    
 0.5524861878453039                 </td><td>0.6221177944862155   
 0.5963018118639861   
 0.6197712418300654   
 0.6684762605815238                  </td><td>0.624171763506626    
 0.5988786952089704    
 0.5933995922528033    
 0.6166539245667686               </td><td>0.6228214195503916    
 0.5966542167540254    
 0.5895691609977324    
 0.5361177028951115           </td></tr>
<tr><td>green_recovery                            </td><td>Glove    
 Distilbert <br>   
 Roberta   <br> 
 Bert     <br>         </td><td>0.994475138121547    
 0.994475138121547    
 0.9834254143646409    
 0.9668508287292817                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972067039106145   
 0.4971590909090909                  </td><td>0.5   <br> 
 0.5    <br>
 0.49444444444444446    
 0.4861111111111111               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4958217270194986    
 0.49157303370786515           </td></tr>
<tr><td>health                                    </td><td>Bert  <br>  
 Distilbert    
 Glove    <br>
 Roberta      <br>        </td><td>0.9613259668508287    
 0.9392265193370166    
 0.9447513812154696    
 0.9558011049723757                 </td><td>0.819047619047619   
 0.6964990138067061   
 0.707687338501292   
 0.7829545454545455                  </td><td>0.7164082687338501    
 0.7574289405684755    
 0.707687338501292    
 0.6608527131782945               </td><td>0.7565802113352544    
 0.7219662058371736    
 0.707687338501292    
 0.7027914614121511           </td></tr>
<tr><td>human_rights                              </td><td>Glove    <br>
 Roberta    <br>
 Bert   <br> 
 Distilbert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.9834254143646409                 </td><td>0.9972222222222222   
 0.9972222222222222   
 0.9972222222222222   
 0.6638576779026217                  </td><td>0.75    <br>
 0.75    <br>
 0.75  <br>  
 0.744413407821229               </td><td>0.8319405756731661    
 0.8319405756731661    
 0.8319405756731661    
 0.6957983193277311           </td></tr>
<tr><td>leaving_no_one_behind                     </td><td>Glove    <br>
 Bert    <br>
 Roberta   <br> 
 Distilbert              </td><td>0.9779005524861878    
 0.9779005524861878    
 0.9668508287292817    
 0.9116022099447514                 </td><td>0.6666666666666666   
 0.6666666666666666   <br>
 0.625   <br>
 0.5555555555555556                  </td><td>0.9888268156424581    
 0.9888268156424581    
 0.9832402234636872    
 0.9553072625698324               </td><td>0.7443502824858756    
 0.7443502824858756    
 0.6914772727272727    
 0.5766081871345029           </td></tr>
<tr><td>multi_stakeholder_collaboration           </td><td>Roberta    <br>
 Bert    <br>
 Distilbert    
 Glove   <br>           </td><td>0.7624309392265194    
 0.7016574585635359    
 0.7292817679558011    
 0.4861878453038674                 </td><td>0.7301989489489489   
 0.6670527670527671   
 0.6792417417417418   
 0.645841001104159                  </td><td>0.6752142857142858    
 0.6854285714285715    
 0.6364285714285715    
 0.6132142857142857               </td><td>0.6888915537434545    
 0.6713075060532687    
 0.6454810728704481    
 0.4810555846718254           </td></tr>
<tr><td>nature_based_solution                     </td><td>Bert  <br>  
 Glove    <br>
 Distilbert   <br> 
 Roberta              </td><td>0.7348066298342542    
 0.6795580110497238    
 0.7071823204419889    
 0.7458563535911602                 </td><td>0.6251082251082252   
 0.6269811320754717   
 0.575114503816794   
 0.5015134837644468                  </td><td>0.6864516129032259    
 0.7170967741935483    
 0.6058064516129033    
 0.5011827956989248               </td><td>0.6339737108190091    
 0.6131338443396227    
 0.5785334563507754    
 0.4993987493987494           </td></tr>
<tr><td>no tag                                    </td><td>Glove  <br>  
 Roberta    <br>
 Bert    <br>
 Distilbert              </td><td>0.856353591160221    
 0.8011049723756906    
 0.7569060773480663    
 0.7734806629834254                 </td><td>0.8191214470284238   
 0.6260319207484866   
 0.5886497064579257   
 0.5796526054590571                  </td><td>0.6062365591397849    
 0.598494623655914    
 0.5974193548387097    
 0.5690322580645162               </td><td>0.6346273291925466    
 0.6082251082251082    
 0.5923423423423423    
 0.5731377624388841           </td></tr>
<tr><td>plastic                                   </td><td>Distilbert    
 Roberta    
 Glove   <br> 
 Bert              </td><td>0.994475138121547    
 0.988950276243094    
 0.9723756906077348    
 0.9668508287292817                 </td><td>0.75   <br>
 0.6666666666666666   
 0.5833333333333334   
 0.5714285714285714                  </td><td>0.9972222222222222    
 0.9944444444444445    
 0.9861111111111112    
 0.9833333333333334               </td><td>0.8319405756731661    
 0.7472067039106145    
 0.635814889336016    
 0.6165254237288136           </td></tr>
<tr><td>poverty_reduction                         </td><td>Glove    <br>
 Bert    <br>
 Distilbert    
 Roberta              </td><td>0.8342541436464088    
 0.7348066298342542    
 0.8121546961325967    
 0.7845303867403315                 </td><td>0.7614583333333333   
 0.6404761904761904   
 0.7213827838827839   
 0.6423670668953688                  </td><td>0.6648836336336337    
 0.682713963963964    
 0.590746996996997    
 0.5934684684684685               </td><td>0.6920372050816697    
 0.6502415458937199    
 0.6055128205128205    
 0.6051350897801645           </td></tr>
<tr><td>public_private_partnership                </td><td>Distilbert    
 Roberta <br>   
 Bert   <br> 
 Glove  <br>            </td><td>0.7900552486187845    
 0.8784530386740331    
 0.8674033149171271    
 0.5966850828729282                 </td><td>0.5253727753727754   
 0.5488636363636363   
 0.5197044334975369   
 0.4889328063241107                  </td><td>0.5342755035737492    
 0.5139701104613386    
 0.5077972709551657    
 0.4727095516569201               </td><td>0.5268299394606495    
 0.5091222879684417    
 0.5027472527472527    
 0.4388192822255256           </td></tr>
<tr><td>sids                                      </td><td>Distilbert <br>   
 Bert    <br>
 Glove    <br>
 Roberta              </td><td>0.9668508287292817    
 0.9502762430939227    
 0.9281767955801105    
 0.9613259668508287                 </td><td>0.7387005649717514   
 0.6579457364341085   
 0.6064560439560439   
 0.6526217228464419                  </td><td>0.660952380952381    
 0.7328571428571429    
 0.7214285714285714    
 0.5776190476190476               </td><td>0.6914772727272727    
 0.6870317002881845    
 0.6389442995243211    
 0.6011960969468051           </td></tr>
<tr><td>south_south_cooperation                   </td><td>Roberta    
 Glove    <br>
 Bert   <br> 
 Distilbert              </td><td>0.9723756906077348    
 0.9779005524861878    
 0.9779005524861878    
 0.9723756906077348                 </td><td>0.6582397003745318   
 0.4889502762430939   
 0.4889502762430939   
 0.4888888888888889                  </td><td>0.6193502824858756 <br>   
 0.5    <br>
 0.5   <br> 
 0.4971751412429379               </td><td>0.6358148893360162    
 0.494413407821229    
 0.494413407821229    
 0.4929971988795518           </td></tr>
<tr><td>structural_system_transformation          </td><td>Roberta    
 Glove    <br>
 Bert    <br>
 Distilbert              </td><td>0.8674033149171271    
 0.8232044198895028    
 0.8176795580110497    
 0.8453038674033149                 </td><td>0.6473684210526316   
 0.5722049689440993   
 0.5497076023391813   
 0.5687869822485208                  </td><td>0.5720411663807891    
 0.5664665523156089    
 0.5437392795883362    
 0.5398799313893654               </td><td>0.5886363636363636    
 0.569047619047619    
 0.5461591064508776    
 0.5455523672883789           </td></tr>
</tbody>
</table>

In [9]:
<table>
<thead>
<tr><th>Category                                  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>Unnamed: 4                                </td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.9337016574585635    
 0.9558011049723757    
 0.9502762430939227    
 0.9392265193370166                 </td><td>0.6076589595375723   
 0.47790055248618785   
 0.4777777777777778   
 0.47752808988764045                  </td><td>0.6076589595375723    
 0.5    
 0.49710982658959535    
 0.4913294797687861               </td><td>0.6076589595375723    
 0.4887005649717514    
 0.48725212464589235    
 0.4843304843304843           </td></tr>
<tr><td>cov_19                                    </td><td>Roberta    
 Bert    
 Distilbert    
 Glove              </td><td>0.9834254143646409    
 0.9779005524861878    
 0.9723756906077348    
 0.9668508287292817                 </td><td>0.49444444444444446   
 0.49441340782122906   
 0.4943820224719101   
 0.4943502824858757                  </td><td>0.4972067039106145    
 0.49441340782122906    
 0.49162011173184356    
 0.4888268156424581               </td><td>0.4958217270194986    
 0.49441340782122906    
 0.4929971988795518    
 0.49157303370786515           </td></tr>
<tr><td>crisis_setting                            </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 0.994475138121547                 </td><td>1.0   
 1.0   
 1.0   
 0.5                  </td><td>1.0    
 1.0    
 1.0    
 0.4972375690607735               </td><td>1.0    
 1.0    
 1.0    
 0.4986149584487534           </td></tr>
<tr><td>digital_transformation                    </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.988950276243094    
 0.988950276243094    
 0.988950276243094    
 0.9779005524861878                 </td><td>0.494475138121547   
 0.494475138121547   
 0.494475138121547   
 0.49441340782122906                  </td><td>0.5    
 0.5    
 0.5    
 0.49441340782122906               </td><td>0.49722222222222223    
 0.49722222222222223    
 0.49722222222222223    
 0.49441340782122906           </td></tr>
<tr><td>food_and_agricultural_commodities_strategy</td><td>Glove    
 Bert    
 Roberta    
 Distilbert              </td><td>0.6464088397790055    
 0.6408839779005525    
 0.6132596685082873    
 0.6408839779005525                 </td><td>0.6588888888888889   
 0.6333292353085813   
 0.6418693982074264   
 0.619824951375382                  </td><td>0.6639908256880733    
 0.6381880733944953    
 0.6411824668705403    
 0.6099006116207951               </td><td>0.6455324357405141    
 0.6333177497272868    
 0.6132478632478632    
 0.6112157562539241           </td></tr>
<tr><td>green_recovery                            </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.994475138121547    
 0.994475138121547    
 0.9834254143646409    
 0.9668508287292817                 </td><td>0.4972375690607735   
 0.4972375690607735   
 0.4972067039106145   
 0.4971590909090909                  </td><td>0.5    
 0.5    
 0.49444444444444446    
 0.4861111111111111               </td><td>0.4986149584487534    
 0.4986149584487534    
 0.4958217270194986    
 0.49157303370786515           </td></tr>
<tr><td>health                                    </td><td>Bert    
 Distilbert    
 Roberta    
 Glove              </td><td>0.9613259668508287    
 0.9392265193370166    
 0.9558011049723757    
 0.9392265193370166                 </td><td>0.819047619047619   
 0.6964990138067061   
 0.7829545454545455   
 0.6024011299435028                  </td><td>0.7164082687338501    
 0.7574289405684755    
 0.6608527131782945    
 0.5468346253229974               </td><td>0.7565802113352544    
 0.7219662058371736    
 0.7027914614121511    
 0.56116376460216           </td></tr>
<tr><td>human_rights                              </td><td>Glove    
 Roberta    
 Bert    
 Distilbert              </td><td>0.994475138121547    
 0.994475138121547    
 0.994475138121547    
 0.9337016574585635                 </td><td>0.9972222222222222   
 0.9972222222222222   
 0.9972222222222222   
 0.5714285714285714                  </td><td>0.75    
 0.75    
 0.75    
 0.9664804469273742               </td><td>0.8319405756731661    
 0.8319405756731661    
 0.8319405756731661    
 0.6076589595375723           </td></tr>
<tr><td>leaving_no_one_behind                     </td><td>Glove    
 Distilbert    
 Bert    
 Roberta              </td><td>1.0    
 0.9779005524861878    
 0.9779005524861878    
 0.9668508287292817                 </td><td>1.0   
 0.6666666666666666   
 0.6666666666666666   
 0.625                  </td><td>1.0    
 0.9888268156424581    
 0.9888268156424581    
 0.9832402234636872               </td><td>1.0    
 0.7443502824858756    
 0.7443502824858756    
 0.6914772727272727           </td></tr>
<tr><td>multi_stakeholder_collaboration           </td><td>Glove    
 Bert    
 Roberta    
 Distilbert              </td><td>0.7734806629834254    
 0.7237569060773481    
 0.7403314917127072    
 0.6353591160220995                 </td><td>0.7414634146341463   
 0.6783010933557612   
 0.6941637630662021   
 0.6385564240351734                  </td><td>0.698    
 0.6817142857142857    
 0.6592142857142858    
 0.6620714285714285               </td><td>0.7113013032483952    
 0.6798953027730618    
 0.6690527134798676    
 0.6228214195503915           </td></tr>
<tr><td>nature_based_solution                     </td><td>Distilbert    
 Glove    
 Bert    
 Roberta              </td><td>0.7182320441988951    
 0.7790055248618785    
 0.7348066298342542    
 0.7679558011049724                 </td><td>0.5670205706702056   
 0.5647321428571428   
 0.5212114337568058   
 0.5219298245614035                  </td><td>0.5868817204301076    
 0.5467741935483871    
 0.5201075268817205    
 0.5145161290322581               </td><td>0.5711498257839721    
 0.5508684863523574    
 0.5205298013245033    
 0.5126923076923077           </td></tr>
<tr><td>no tag                                    </td><td>Bert    
 Glove    
 Roberta    
 Distilbert              </td><td>0.8232044198895028    
 0.8287292817679558    
 0.8121546961325967    
 0.7734806629834254                 </td><td>0.6758392955421024   
 0.6781437125748503   
 0.6327116212338594   
 0.5694002123142251                  </td><td>0.6374193548387097    
 0.5895698924731183    
 0.5795698924731183    
 0.556236559139785               </td><td>0.6517556517556518    
 0.6066596565019278    
 0.5916932059447982    
 0.560497482973053           </td></tr>
<tr><td>plastic                                   </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.994475138121547    
 0.988950276243094    
 0.9779005524861878    
 0.9558011049723757                 </td><td>0.75   
 0.6666666666666666   
 0.6   
 0.5555555555555556                  </td><td>0.9972222222222222    
 0.9944444444444445    
 0.9888888888888889    
 0.9777777777777779               </td><td>0.8319405756731661    
 0.7472067039106145    
 0.6610486891385767    
 0.5886363636363636           </td></tr>
<tr><td>poverty_reduction                         </td><td>Glove    
 Roberta    
 Distilbert    
 Bert              </td><td>0.7790055248618785    
 0.7569060773480663    
 0.712707182320442    
 0.6906077348066298                 </td><td>0.6852941176470588   
 0.6536796536796536   
 0.6143578643578644   
 0.6054644808743169                  </td><td>0.7305743243243243    
 0.6865615615615616    
 0.6487424924924925    
 0.6448948948948949               </td><td>0.6997345719973458    
 0.6644759015840918    
 0.6210950080515298    
 0.6082251082251081           </td></tr>
<tr><td>public_private_partnership                </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.861878453038674    
 0.861878453038674    
 0.850828729281768    
 0.8950276243093923                 </td><td>0.5104768786127167   
 0.5104768786127167   
 0.49736842105263157   
 0.44751381215469616                  </td><td>0.5047108512020793    
 0.5047108512020793    
 0.4985380116959064    
 0.5               </td><td>0.49972360420121614    
 0.49972360420121614    
 0.49394221808014915    
 0.47230320699708456           </td></tr>
<tr><td>sids                                      </td><td>Bert    
 Distilbert    
 Glove    
 Roberta              </td><td>0.9502762430939227    
 0.9613259668508287    
 0.9668508287292817    
 0.9613259668508287                 </td><td>0.6579457364341085   
 0.6886363636363637   
 0.7360335195530726   
 0.6526217228464419                  </td><td>0.7328571428571429    
 0.6580952380952381    
 0.5804761904761905    
 0.5776190476190476               </td><td>0.6870317002881845    
 0.6718466718466718    
 0.6165254237288136    
 0.6011960969468051           </td></tr>
<tr><td>south_south_cooperation                   </td><td>Bert    
 Distilbert    
 Roberta    
 Glove              </td><td>0.9779005524861878    
 0.9723756906077348    
 0.9723756906077348    
 0.9392265193370166                 </td><td>0.4889502762430939   
 0.4888888888888889   
 0.4888888888888889   
 0.4885057471264368                  </td><td>0.5    
 0.4971751412429379    
 0.4971751412429379    
 0.480225988700565               </td><td>0.494413407821229    
 0.4929971988795518    
 0.4929971988795518    
 0.48433048433048437           </td></tr>
<tr><td>structural_system_transformation          </td><td>Glove    
 Roberta    
 Bert    
 Distilbert              </td><td>0.8232044198895028    
 0.7790055248618785    
 0.8121546961325967    
 0.6740331491712708                 </td><td>0.6182306255835668   
 0.5700280112044818   
 0.5440993788819876   
 0.5302308802308803                  </td><td>0.6447970268724986    
 0.6000571755288737    
 0.5405946255002859    
 0.55989136649514               </td><td>0.6287179487179487    
 0.5774976657329599    
 0.5421130952380953    
 0.5133743449532924           </td></tr>
</tbody>
</table>

SyntaxError: invalid syntax (<ipython-input-9-e524c6121540>, line 1)